In [284]:
import json
import pandas as pd
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import Optional, List
from pydantic import BaseModel
from pydantic import Field

In [285]:
with open('../key/key.json') as f:
    k = json.load(f)['key']

In [391]:
# create one time list of 10 human emotions
client = OpenAI(api_key=k)
class Chr_answer(BaseModel):
    Characteristics: List[str] = Field(None, description="Emotions as a list of strings")

#few shot prompt with api
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "system", "content": "Find 50 non-redundant human emotions. These emotions should be different from one another, meaning that for example, of the emotions 'joy' and 'happiness', only pick one. Of 'Shame' and 'Embarrassment' and of 'Envy' and 'Jealousy', only pick one. So on and so forth."},
        {"role": "system", "content": "Stop finding if you've already found 50."},
        {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
        {"role": "user", "content": "Give me about 50 emotions addressing non-redundant and different human feelings."}
    ],
    response_format=Chr_answer,
)

#you can also proceed from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
emotions = out
print(emotions)
print(len(emotions))

['Joy', 'Sadness', 'Anger', 'Fear', 'Disgust', 'Surprise', 'Love', 'Gratitude', 'Loneliness', 'Satisfaction', 'Awe', 'Amusement', 'Boredom', 'Calmness', 'Confusion', 'Confidence', 'Contentment', 'Curiosity', 'Desperation', 'Disappointment', 'Determination', 'Doubt', 'Empathy', 'Excitement', 'Frustration', 'Guilt', 'Hatred', 'Hope', 'Hurt', 'Impatience', 'Insecurity', 'Irritation', 'Longing', 'Nostalgia', 'Optimism', 'Pride', 'Relief', 'Remorse', 'Resentment', 'Shame', 'Sympathy', 'Tenderness', 'Triumph', 'Trust', 'Vulnerability', 'Wonder', 'Zeal']
47


In [392]:
#List 10 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."
client = OpenAI(api_key=k)

Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) #MyEnumType = Enum('MyEnumType', myEnumStrings)

class BrandAttributes(BaseModel):
    brand_name: str = Field(None, description=" name as str")
    characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")

class BrandChars(BaseModel):
    a: List[BrandAttributes] = Field(None, description="List of BrandAttributes")
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 30 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."}
    ],
    response_format=BrandChars,
)

out = json.loads(completion.choices[0].message.content)
brand_emotions = out
out = list(out.values())[0]
df_brand = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','characteristic'])
df_brand

,Name,characteristic
0,Nike,"[Confidence, Insecurity]"
1,Under Armour,"[Determination, Zeal]"
2,Levi's,"[Nostalgia, Pride]"
3,Ralph Lauren,"[Pride, Contentment]"
4,Calvin Klein,"[Confidence, Love]"
5,Tommy Hilfiger,"[Trust, Confidence]"
6,Gap,"[Nostalgia, Contentment]"
7,Old Navy,"[Joy, Contentment]"
8,Coach,"[Love, Pride]"
9,Michael Kors,"[Confidence, Pride]"


In [393]:
#Take the example song, and list which one of the emotions in the given list are most closely associated with the song.")

client = OpenAI(api_key=k)

Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) 

class SongAttributes(BaseModel):
    song_name: str
    song_lyrics_summary :str = Field(None, description="Summary as str; Don't state the exact lyrics.")
    characteristics: List[Characteristic]

class SongsChars(BaseModel):
    a: List[SongAttributes] = Field(None, description="List of song attributes.")

prompt = """
As prompt, take the song 'Black Balloon', list summary of its lyrics, and a list of the emotions from the given list that are most closely associated with the song.
"""

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    response_format=SongsChars,
)
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
df_song = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','lyrics_summary', 'characteristic'])
df_song

,Name,lyrics_summary,characteristic
0,Black Balloon,The song narrates the struggles of watching so...,"[Sadness, Hope, Desperation, Love, Hurt]"


In [405]:
#MultilabelBinarize brands and song
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=emotions)

df_brand2 = pd.DataFrame(mlb.fit_transform(df_brand['characteristic']), columns = emotions)
df_brand_mb = pd.merge(df_brand2, df_brand[['Name']], how ='left', left_index=True, right_index=True)
df_brand_mb.head()


,Joy,Sadness,Anger,Fear,Disgust,Surprise,Love,Gratitude,Loneliness,Satisfaction,...,Resentment,Shame,Sympathy,Tenderness,Triumph,Trust,Vulnerability,Wonder,Zeal,Name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Nike
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Under Armour
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Levi's
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Ralph Lauren
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Calvin Klein


In [408]:
mlb = MultiLabelBinarizer(classes=emotions)

df_song2 = pd.DataFrame(mlb.fit_transform(df_song['characteristic']), columns = emotions)
df_song_mb = pd.merge(df_song2, df_song[['Name']], how ='left', left_index=True, right_index=True)
df_song_mb.head()

#song_e = list(song_emotions[0].values())[2]

#df_song = pd.DataFrame(mlb.fit_transform([song_e]), columns = emotions, index=[list(song_emotions[0].keys())[0]])
#df_song

,Joy,Sadness,Anger,Fear,Disgust,Surprise,Love,Gratitude,Loneliness,Satisfaction,...,Resentment,Shame,Sympathy,Tenderness,Triumph,Trust,Vulnerability,Wonder,Zeal,Name
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Black Balloon


In [409]:
df_b_T = df_brand_mb.set_index('Name').T
df_s_T = df_song_mb.set_index('Name').T

df_s_T

Name,Black Balloon
Joy,0
Sadness,1
Anger,0
Fear,0
Disgust,0
Surprise,0
Love,1
Gratitude,0
Loneliness,0
Satisfaction,0


In [411]:
#no correlation, makes sense as this song is mainly sad
df_b_T.corrwith(df_s_T, axis = 1)

Joy              NaN
Sadness          NaN
Anger            NaN
Fear             NaN
Disgust          NaN
Surprise         NaN
Love             NaN
Gratitude        NaN
Loneliness       NaN
Satisfaction     NaN
Awe              NaN
Amusement        NaN
Boredom          NaN
Calmness         NaN
Confusion        NaN
Confidence       NaN
Contentment      NaN
Curiosity        NaN
Desperation      NaN
Disappointment   NaN
Determination    NaN
Doubt            NaN
Empathy          NaN
Excitement       NaN
Frustration      NaN
Guilt            NaN
Hatred           NaN
Hope             NaN
Hurt             NaN
Impatience       NaN
Insecurity       NaN
Irritation       NaN
Longing          NaN
Nostalgia        NaN
Optimism         NaN
Pride            NaN
Relief           NaN
Remorse          NaN
Resentment       NaN
Shame            NaN
Sympathy         NaN
Tenderness       NaN
Triumph          NaN
Trust            NaN
Vulnerability    NaN
Wonder           NaN
Zeal             NaN
dtype: float6